In [23]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("t00.csv",index_col=0,parse_dates=True)

In [3]:
df["volume_ratio"] = df["volume"]/df["volume_mean"]
df["volume_ratio3"] = df["volume"].rolling(3).mean()/df["volume_mean"].rolling(3).mean()
df["volume_ratio5"] = df["volume"].rolling(5).mean()/df["volume_mean"].rolling(5).mean()
df["minchange"] = df["price"].pct_change()
df["minchange3"] = df["price"].pct_change(3)
df["minchange5"] = df["price"].pct_change(5)
df["momentum"] = df["minchange"] * df["volume_ratio"]
df["momentum3"] = df["minchange3"] * df["volume_ratio3"]
df["momentum5"] = df["minchange5"] * df["volume_ratio5"]

In [24]:
df.tail(3)

,change,price,volume,volume_mean,volume_ratio,volume_ratio3,volume_ratio5,minchange,minchange3,minchange5,momentum,momentum3,momentum5,wave_change,wave_volume_ratio,wave_result,min_result3,min_result5,momentum_wave
date,,,,,,,,,,,,,,,,,,,
2016-12-30 14:57:00,1.000528,6473.794,569100.0,464070.0,1.226324,1.196813,1.219388,0.000005,0.000280,-0.000037,0.000006,0.000335,-0.000045,1.00028,1.196813,1.0,NaN,NaN,1.197148
2016-12-30 14:58:00,1.000528,6473.794,0.0,0.0,NaN,1.211054,1.197964,0.000000,0.000141,0.000016,NaN,0.000170,0.000019,1.00000,NaN,NaN,NaN,NaN,NaN
2016-12-30 14:59:00,1.000528,6473.794,0.0,0.0,NaN,1.226324,1.196813,0.000000,0.000005,0.000280,NaN,0.000006,0.000335,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df = df.reset_index()
df.columns = ["date"] + list(df.columns)[1:]

In [108]:
sub_df = df[:480]
df = sub_df

In [16]:
wave_change_se = pd.Series(index=df.index)
wave_volume_se = pd.Series(index=df.index)
wave_result_se = pd.Series(index=df.index)
min_result3_se = pd.Series(index=df.index)
min_result5_se = pd.Series(index=df.index)
# before_se = pd.Series(index=df.index)
# after_se = pd.Series(index=df.index)
# close_result_se = pd.Series()

min_change_list = list(df["minchange"])
date_list = list(df["date"])
price_list = list(df["price"])
for index,row in df.iterrows():
    date = row["date"].strftime("%H:%M")
    if date == "09:30" or date == "14:59":
        continue
        
    pct_change = row["minchange"]
    
    #回溯
    idx = index
    while min_change_list[idx] * pct_change > 0 and date_list[idx].strftime("%H:%M") != "14:59" and idx >= 0:
        idx -= 1
        
    before_se[index] = idx
    
    wave_change_se[index] = row["price"] / price_list[idx]

    sub_df = df[idx+1:index+1]
    
    if len(sub_df) == 0 or sub_df["volume_mean"].sum() == 0:
        continue
    wave_volume_se[index] = sub_df["volume"].sum()/sub_df["volume_mean"].sum()
    
    #未来
    idx = index
    while date_list[idx].strftime("%H:%M") != "14:59" and min_change_list[idx] * pct_change > 0:
        idx += 1
        
    after_se[index] = idx
    wave_result_se[index] = price_list[idx] / row["price"] #未来价除以现价
    if date < "14:57":
        min_result3_se[index] = price_list[index+3] / row["price"]
    if date < "14:55":
        min_result5_se[index] = price_list[index+5] / row["price"]
    
    if index % 1000 == 0:
        print index
    
# df["before_idx"] = before_se
# df["after_idx"] = after_se
df["wave_change"] = wave_change_se
df["wave_volume_ratio"] = wave_volume_se
df["wave_result"] = wave_result_se
df["min_result3"] = min_result3_se
df["min_result5"] = min_result5_se
df = df.set_index(["date"])
# df.to_csv("t00.csv")

3000
5000
7000
8000
9000
10000
11000
13000
14000
15000
16000
17000
19000
20000
21000
22000
23000
25000
26000
27000
28000
29000
31000
32000
33000
34000
35000
37000
38000
39000
40000
41000
43000
44000
45000
46000
47000
49000
50000
51000
52000
53000
55000
56000
57000
58000
59000
61000
62000
63000
64000
65000
67000
68000
69000
70000
71000
73000
74000
75000
76000
77000


In [17]:
df.to_csv("t00.csv")

In [19]:
df.tail(10)

,change,price,volume,volume_mean,volume_ratio,volume_ratio3,volume_ratio5,minchange,minchange3,minchange5,momentum,momentum3,momentum5,wave_change,wave_volume_ratio,wave_result,min_result3,min_result5
date,,,,,,,,,,,,,,,,,,
2016-12-30 14:50:00,1.000650,6474.581,6088400.0,6126570.0,0.993770,1.106873,1.105663,-0.000055,0.000635,0.001153,-0.000055,0.000702,0.001274,0.999945,0.993770,1.000179,0.999862,0.999738
2016-12-30 14:51:00,1.000829,6475.740,7212200.0,6815600.0,1.058190,1.075678,1.098110,0.000179,0.000350,0.001129,0.000189,0.000377,0.001240,1.000179,1.058190,0.999737,0.999420,0.999694
2016-12-30 14:52:00,1.000566,6474.035,6832400.0,7035910.0,0.971076,1.007754,1.066393,-0.000263,-0.000139,0.000550,-0.000256,-0.000141,0.000587,0.999737,0.971076,0.999822,0.999822,0.999963
2016-12-30 14:53:00,1.000512,6473.690,9538800.0,7366490.0,1.294891,1.111481,1.102022,-0.000053,-0.000138,0.000034,-0.000069,-0.000153,0.000037,0.999683,1.136699,0.999875,1.000011,1.000016
2016-12-30 14:54:00,1.000248,6471.981,9567400.0,7969090.0,1.200564,1.159449,1.111162,-0.000264,-0.000580,-0.000457,-0.000317,-0.000673,-0.000507,0.999420,1.159449,1.000140,1.000280,1.000280
2016-12-30 14:55:00,1.000388,6472.884,9718000.0,8235990.0,1.179943,1.222838,1.145518,0.000140,-0.000178,-0.000262,0.000165,-0.000217,-0.000300,1.000140,1.179943,1.000141,1.000141,NaN
2016-12-30 14:56:00,1.000523,6473.761,11245400.0,9291480.0,1.210292,1.197448,1.175519,0.000135,0.000011,-0.000306,0.000164,0.000013,-0.000359,1.000275,1.196031,1.000005,1.000005,NaN
2016-12-30 14:57:00,1.000528,6473.794,569100.0,464070.0,1.226324,1.196813,1.219388,0.000005,0.000280,-0.000037,0.000006,0.000335,-0.000045,1.000280,1.196813,1.000000,NaN,NaN
2016-12-30 14:58:00,1.000528,6473.794,0.0,0.0,NaN,1.211054,1.197964,0.000000,0.000141,0.000016,NaN,0.000170,0.000019,1.000000,NaN,NaN,NaN,NaN
